In [8]:
# Data Container imports
from class_DataContainer import DataContainer
import pickle

# Data manipulation imports
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Flatten, Reshape, Conv1D, MaxPooling1D
from tensorflow.data import Dataset


with open('data.pkl', 'rb') as f:
    data = pickle.load(f)


In [2]:
X_train, X_val, y_train, y_val = train_test_split(data.feature, data.label, test_size=0.2, random_state=99)

train_data = Dataset.from_tensor_slices((X_train, y_train))
val_data = Dataset.from_tensor_slices((X_val, y_val))

train_data = train_data.shuffle(buffer_size=1024).batch(64)
val_data = val_data.batch(64)

data.feature.shape
layers = [InputLayer(input_shape=(None,64,17)),
          Dense(64, activation='relu'),
          Reshape((64,1)),
          Conv1D(64, (3), activation='relu'), 
          Dense(32, activation='relu'),
          Conv1D(32, (3), activation='relu'),
          MaxPooling1D(3),
          Dense(8, activation='relu'), 
          Conv1D(8, (3), activation='relu'),
          Dense(2, activation='relu'),
          Conv1D(2, (3), activation='relu'),
          Flatten(),
          Dense(1, activation='sigmoid')]
model = Sequential(layers=layers)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [3]:
history = model.fit(train_data, validation_data=val_data, epochs=50, verbose=2)

Epoch 1/50
562/562 - 4s - loss: 0.4364 - accuracy: 0.8040 - val_loss: 0.1075 - val_accuracy: 0.9772 - 4s/epoch - 6ms/step
Epoch 2/50
562/562 - 2s - loss: 0.1144 - accuracy: 0.9687 - val_loss: 0.1173 - val_accuracy: 0.9715 - 2s/epoch - 4ms/step
Epoch 3/50
562/562 - 2s - loss: 0.1194 - accuracy: 0.9680 - val_loss: 0.5969 - val_accuracy: 0.8172 - 2s/epoch - 4ms/step
Epoch 4/50
562/562 - 2s - loss: 0.0937 - accuracy: 0.9743 - val_loss: 0.0542 - val_accuracy: 0.9856 - 2s/epoch - 4ms/step
Epoch 5/50
562/562 - 2s - loss: 0.0635 - accuracy: 0.9819 - val_loss: 0.0599 - val_accuracy: 0.9853 - 2s/epoch - 4ms/step
Epoch 6/50
562/562 - 2s - loss: 0.0650 - accuracy: 0.9816 - val_loss: 0.0542 - val_accuracy: 0.9861 - 2s/epoch - 4ms/step
Epoch 7/50
562/562 - 2s - loss: 0.0624 - accuracy: 0.9826 - val_loss: 0.0654 - val_accuracy: 0.9826 - 2s/epoch - 4ms/step
Epoch 8/50
562/562 - 2s - loss: 0.0604 - accuracy: 0.9833 - val_loss: 0.3856 - val_accuracy: 0.9314 - 2s/epoch - 4ms/step
Epoch 9/50
562/562 - 2s 

In [13]:
pred_raw = model.predict(X_val)
pred = np.round(pred_raw)
accuracy = np.mean(pred == y_val)
print(f"Keras Sequential Accuracy: {100 * accuracy:.2f}%")
model.save('model.keras')


281/281 [==============================] - 0s 1ms/step
Keras Sequential Accuracy: 99.48%
